# Self-Driving Car Engineer Nanodegree
## Term 1, Project 1: **Finding Lane Lines on the Road**
### Jupyter Notebook for Project: CarND-LaneLines-P1

In [1]:
# Import packages, and setup associated shorthand names.
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
%matplotlib inline
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [62]:
# Define helpful functions
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=5):
    """
    Average/extrapolate the line segments you detect to map out the full
    extent of the lane. Separate line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.

    NOTE: The input parameter "lines" is a structure array, whose format
    matches that of the return value (of an earlier call to)
    cv2.HoughLinesP.
    "lines" length equals the number of merged line segments found.
    Each element of the structure array has these fields: 
      point1    End-point of the line segment; two-element vector
      point2    End-point of the line segment; two-element vector
      length    Distance between point1 and point2
      theta     Angle (in degrees) of the Hough transform bin
      rho       Rho-axis position of the Hough transform bin
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """  
    # initialize working variables; for calculating average slope, & upper-most(on screen) y coords
    right_x1_total = 0.0; right_x2_total = 0.0; right_y1_total = 0.0; right_y2_total = 0.0
    left_x1_total = 0.0; left_x2_total = 0.0; left_y1_total = 0.0; left_y2_total = 0.0
    right_line_count = 0; left_line_count = 0 # number of lines to be averaged for right and left lanes
    ysize = img.shape[0] # Pull out the y size of the image
    right_lowest_y1 = ysize-160 # lowest y1 - upper-most(on screen) y coord found in all the left lane lines
    left_lowest_y1 = ysize-160  # lowest y1 - upper-most(on screen) y coord found in all the right lane lines
    
    # Prepare to calculate average slope of right and left lane lines
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope=(y2-y1)/(x2-x1) # slope of this line
            if (slope>0.0):
                # process a right lane line, as long as it isn't an outlier
                if x1 > 480:
                    right_line_count += 1
                    right_x1_total += x1; right_x2_total += x2; right_y1_total += y1; right_y2_total += y2
                    if y1 < right_lowest_y1:
                        right_lowest_y1 = y1
            else:
                # process a left lane line, as long as it isn't an outlier
                if x1 < 470:
                    # process a left lane line
                    left_line_count += 1
                    left_x1_total += x1; left_x2_total += x2; left_y1_total += y1; left_y2_total += y2
                    if y1 < left_lowest_y1:
                        left_lowest_y1 = y1
    
    if right_line_count > 0:
        # calculate average (x1,y1)(x2,y2) coords of right lane lines
        right_x1_avg = int(right_x1_total/right_line_count)
        right_y1_avg = int(right_y1_total/right_line_count)
        right_x2_avg = int(right_x2_total/right_line_count)
        right_y2_avg = int(right_y2_total/right_line_count)
        right_avg_slope = (right_y2_avg-right_y1_avg)/(right_x2_avg-right_x1_avg)
        # Figure out (extrapolate) the lower (on screen; higher numerically) xcoords of the lane lines
        # (by solving for x2 in the slope equation for a line)
        right_lower_xcoord = int((ysize - right_y1_avg)/right_avg_slope + right_x1_avg)
        right_upper_xcoord = int(-((right_y2_avg - right_lowest_y1)/right_avg_slope - right_x2_avg))
        # Draw extrapolated right and left lane lines
        color=[255, 0, 0]
        cv2.line(img, (right_upper_xcoord, right_lowest_y1), (right_lower_xcoord, ysize), color, thickness)
        
    if left_line_count > 0:   
        left_x1_avg = int(left_x1_total/left_line_count)
        left_y1_avg = int(left_y1_total/left_line_count)
        left_x2_avg = int(left_x2_total/left_line_count)
        left_y2_avg = int(left_y2_total/left_line_count) 
        left_avg_slope = (left_y2_avg-left_y1_avg)/(left_x2_avg-left_x1_avg)
        left_lower_xcoord = int((ysize - left_y1_avg)/left_avg_slope + left_x1_avg)
        left_upper_xcoord = int(-((left_y2_avg - left_lowest_y1)/left_avg_slope - left_x2_avg))
        color=[255, 0, 0]    
        cv2.line(img, (left_upper_xcoord, left_lowest_y1), (left_lower_xcoord, ysize), color, thickness)
    
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

In [78]:
# Define the function containing the Lane Detection Pipeline
def process_image(img):
    """
    `img` should be the initial image.
    
    The result is an image with highlighted lane lines overlayed
    on the initial image.
    """
    # convert the initial image to grayscale
    gray = grayscale(img)

    # Apply Gaussian smoothing with kernel size of 5, on the gray scale image,
    # to create blurred Gray Scale image
    blur_gray = cv2.GaussianBlur(gray,(5, 5),0)

    # Apply Canny transform with low_threshold of 45, and high_threshold of 160,
    # on the blurred Gray Scale image, to create a blurred Canny transformed image
    edges = cv2.Canny(blur_gray, 45, 160)

    # Define a four sided polygon to mask
    imshape = img.shape
    vertices = np.array([[(40,imshape[0]),(440, 324), (510, 324), (imshape[1]-30,imshape[0])]], dtype=np.int32)
    
    # Create an image with edges masked, from the blurred Canny transformed image
    masked_img = region_of_interest(edges, vertices)

    # Create a line image, to be used to overlay on original image, using function hough_lines,
    # with parameters: rho=2, theta=np.pi/180, threshold=3, min_line_len=11, max_line_gap=7
    line_img = hough_lines(masked_img, 2, np.pi/180, 3, 12, 6)

    # Overlay the line image on top of the original image, to create the final image
    return weighted_img(line_img, img, α=0.8, β=1., λ=0.)

## process_image pipeline function Test Images

In [91]:
# Perform tests of the process_image pipeline function,
# using all images from the test_images directory,
# saving all output files to the test_images_output directory;
# as instructed in the Test Images & Build a Lane Finding Pipeline sections
# of the CarND-LaneLines-P1-master/P1.ipynb file.
image = mpimg.imread('test_images/solidYellowleft.jpg')
final_img = process_image(image)
mpimg.imsave('test_images_output/solidYellowleft.jpg',final_img)
image = mpimg.imread('test_images/solidWhiteRight.jpg')
final_img = process_image(image)
mpimg.imsave('test_images_output/solidWhiteRight.jpg',final_img)
image = mpimg.imread('test_images/SolidWhiteCurve.jpg')
final_img = process_image(image)
mpimg.imsave('test_images_output/solidWhiteCurve.jpg',final_img)
image = mpimg.imread('test_images/SolidYellowCurve.jpg')
final_img = process_image(image)
mpimg.imsave('test_images_output/solidYellowCurve.jpg',final_img)
image = mpimg.imread('test_images/SolidYellowCurve2.jpg')
final_img = process_image(image)
mpimg.imsave('test_images_output/solidYellowCurve2.jpg',final_img)
image = mpimg.imread('test_images/whiteCarLaneSwitch.jpg')
final_img = process_image(image)
mpimg.imsave('test_images_output/whiteCarLaneSwitch.jpg',final_img)

## process_image pipeline function Video Tests

In [81]:
# Now test process_image with solidWhiteRight.mp4 video input file;
# and write out the resulting video file to 'test_videos_output' directory
white_output = 'test_videos_output/solidWhiteRight.mp4'
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


100%|██████████████████████████████████████████████████████████████████████████████▋| 221/222 [00:01<00:00, 111.45it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

Wall time: 2.26 s


In [82]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [92]:
# Now test process_image with solidWhiteRight.mp4 video input file;
# and write out the resulting video file to 'test_videos_output' directory
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
clip2 = VideoFileClip("test_videos/solidYellowLeft.mp4")
yellow_clip = clip2.fl_image(process_image) #NOTE: this function expects color images!!
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


100%|██████████████████████████████████████████████████████████████████████████████▉| 681/682 [00:06<00:00, 110.08it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

Wall time: 6.46 s


In [93]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))